# Spark: Getting Started


## Step 0: Prerequisites & Installation

Follow [these instructions](https://docs.databricks.com/notebooks/notebooks-manage.html#import-a-notebook) to import this notebook into Databricks

Run these commands in your terminal (just once) if you want to run Spark locally.

 * These instructions require a Mac with [Anaconda3](https://anaconda.com/) and [Homebrew](https://brew.sh/) installed.
 * Useful for small data only. For larger data, try [Databricks](https://databricks.com/).

```bash
# Make Homebrew aware of old versions of casks
brew tap caskroom/versions

# Install Java 1.8 (OpenJDK 8)
brew cask install adoptopenjdk8

# Install the current version of Spark
brew install apache-spark

# Install Py4J (connects PySpark to the Java Virtual Machine)
pip install py4j

# Add JAVA_HOME to .bash_profile (makes Java 1.8 your default JVM)
echo "\n# Apache Spark\nexport JAVA_HOME=$(/usr/libexec/java_home -v 1.8)" >> ~/.bash_profile

# Add SPARK_HOME to .bash_profile
echo "\nexport SPARK_HOME=/usr/local/Cellar/apache-spark/2.4.5/libexec" >> ~/.bash_profile

# Add PySpark to PYTHONPATH in .bash_profile
echo "\nexport PYTHONPATH=$SPARK_HOME/python:$PYTHONPATH" >> ~/.bash_profile

# Update current environment
source ~/.bash_profile

```

## Step 1: Create a SparkSession with a SparkContext

In [3]:
import pyspark

# un-comment the following lines if you are running Spark locally
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [4]:
spark

In [5]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

If we need a broadcast (i.e. global) variable, we can declare it like so:

In [6]:
glob = sc.broadcast(list(range(1, 3)))
glob.value

[1, 2]

## Step 2: Download some Amazon reviews (Toys & Games)

In [8]:
# Data is already in the repo, but you can also get it with
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Toys_and_Games_5.json.gz
!gunzip reviews_Toys_and_Games_5.json.gz

--2020-07-23 09:08:29--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Toys_and_Games_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42057555 (40M) [application/x-gzip]
Saving to: ‘reviews_Toys_and_Games_5.json.gz’

reviews_Toys_and_Ga 100%[===================>]  40.11M  6.28MB/s    in 6.3s    

2020-07-23 09:08:36 (6.39 MB/s) - ‘reviews_Toys_and_Games_5.json.gz’ saved [42057555/42057555]



Follow [these instructions](https://docs.databricks.com/data/data.html#import-data-1) to import `reviews_Toys_and_Games_5.json` into Databricks

## Step 3: Create a Spark DataFrame

In [10]:
# this file path will be different if you are running Spark locally
df = spark.read.json('reviews_Toys_and_Games_5.json')

In [11]:
df.persist()

DataFrame[asin: string, helpful: array<bigint>, overall: double, reviewText: string, reviewTime: string, reviewerID: string, reviewerName: string, summary: string, unixReviewTime: bigint]

This last command, `.persist()`, simply stores the DataFrame in memory. See [this page](https://unraveldata.com/to-cache-or-not-to-cache/). It is similar to `.cache()`, but actually more flexible than the latter since you can specify which storage level you want. See [here](https://stackoverflow.com/questions/26870537/what-is-the-difference-between-cache-and-persist).

In [12]:
type(df)

pyspark.sql.dataframe.DataFrame

In [13]:
df.show(5) # default of 20 lines

+----------+-------+-------+--------------------+-----------+--------------+--------------+--------------------+--------------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|  reviewerName|             summary|unixReviewTime|
+----------+-------+-------+--------------------+-----------+--------------+--------------+--------------------+--------------+
|0439893577| [0, 0]|    5.0|I like the item p...|01 29, 2014|A1VXOAVRGKGEAK|         Angie|      Magnetic board|    1390953600|
|0439893577| [1, 1]|    4.0|Love the magnet e...|03 28, 2014| A8R62G708TSCM|       Candace|it works pretty g...|    1395964800|
|0439893577| [1, 1]|    5.0|Both sides are ma...|01 28, 2013|A21KH420DK0ICA|capemaychristy|          love this!|    1359331200|
|0439893577| [0, 0]|    5.0|Bought one a few ...| 02 8, 2014| AR29QK6HPFYZ4|          dcrm|   Daughters love it|    1391817600|
|0439893577| [1, 1]|    4.0|I have a stainles...| 05 5, 2014| ACCH8EOML6FN5|          DoyZ|Great to have

In [14]:
pdf = df.limit(5).toPandas()
pdf

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0439893577,"[0, 0]",5.0,I like the item pricing. My granddaughter want...,"01 29, 2014",A1VXOAVRGKGEAK,Angie,Magnetic board,1390953600
1,0439893577,"[1, 1]",4.0,Love the magnet easel... great for moving to d...,"03 28, 2014",A8R62G708TSCM,Candace,it works pretty good for moving to different a...,1395964800
2,0439893577,"[1, 1]",5.0,Both sides are magnetic. A real plus when you...,"01 28, 2013",A21KH420DK0ICA,capemaychristy,love this!,1359331200
3,0439893577,"[0, 0]",5.0,Bought one a few years ago for my daughter and...,"02 8, 2014",AR29QK6HPFYZ4,dcrm,Daughters love it,1391817600
4,0439893577,"[1, 1]",4.0,I have a stainless steel refrigerator therefor...,"05 5, 2014",ACCH8EOML6FN5,DoyZ,Great to have so he can play with his alphabet...,1399248000


In [15]:
type(pdf)

pandas.core.frame.DataFrame

In [16]:
df.count()

167597

In [17]:
df.columns

['asin',
 'helpful',
 'overall',
 'reviewText',
 'reviewTime',
 'reviewerID',
 'reviewerName',
 'summary',
 'unixReviewTime']

In [18]:
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



The 'nullable = true' bit means that the relevant column tolerates null values.

In [19]:
df.describe().show()

+-------+--------------------+------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+
|summary|                asin|           overall|          reviewText|reviewTime|          reviewerID|        reviewerName|             summary|      unixReviewTime|
+-------+--------------------+------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+
|  count|              167597|            167597|              167597|    167597|              167597|              166759|              167597|              167597|
|   mean|2.1290876613696043E9| 4.356307093802395|                null|      null|                null|   474.1111111111111|               600.0|1.3487585072095563E9|
| stddev|2.1305790004603045E9|0.9935012992132014|                null|      null|                null|   517.0615833762786|                 NaN| 6.089580845001178E7|
|   

In [20]:
df.describe('overall').show()

+-------+------------------+
|summary|           overall|
+-------+------------------+
|  count|            167597|
|   mean| 4.356307093802395|
| stddev|0.9935012992132014|
|    min|               1.0|
|    max|               5.0|
+-------+------------------+



In [21]:
reviews_df = df[['asin', 'overall']]

In [22]:
reviews_df.show()

+----------+-------+
|      asin|overall|
+----------+-------+
|0439893577|    5.0|
|0439893577|    4.0|
|0439893577|    5.0|
|0439893577|    5.0|
|0439893577|    4.0|
|0439893577|    3.0|
|0439893577|    3.0|
|0439893577|    5.0|
|0439893577|    4.0|
|0439893577|    3.0|
|0439893577|    5.0|
|0439893577|    5.0|
|0439893577|    3.0|
|0439893577|    5.0|
|0439893577|    5.0|
|0439893577|    5.0|
|0439893577|    5.0|
|048645195X|    5.0|
|048645195X|    4.0|
|048645195X|    5.0|
+----------+-------+
only showing top 20 rows



In [23]:
def show(df, n=5):
    return df.limit(n).toPandas()

In [24]:
show(reviews_df)

,asin,overall
0,0439893577,5.0
1,0439893577,4.0
2,0439893577,5.0
3,0439893577,5.0
4,0439893577,4.0


In [25]:
reviews_df.count()

167597

In [26]:
sorted_review_df = reviews_df.sort('overall')

In [27]:
show(sorted_review_df)

,asin,overall
0,B008FD8ETS,1.0
1,B008FTF9PO,1.0
2,B008FD8ETS,1.0
3,B008FD1RL0,1.0
4,B008FD8ETS,1.0


In [28]:
import pyspark.sql.functions as F

In [29]:
counts = reviews_df.agg(F.countDistinct('overall'))

In [30]:
counts.show()

+-----------------------+
|count(DISTINCT overall)|
+-----------------------+
|                      5|
+-----------------------+



In [31]:
query = """
SELECT overall, COUNT(*)
FROM reviews
GROUP BY overall
ORDER BY overall
"""

In [32]:
reviews_df.createOrReplaceTempView('reviews')

In [33]:
output = spark.sql(query)

In [34]:
show(output)

,overall,count(1)
0,1.0,4707
1,2.0,6298
2,3.0,16357
3,4.0,37445
4,5.0,102790


In [35]:
output.collect()

[Row(overall=1.0, count(1)=4707),
 Row(overall=2.0, count(1)=6298),
 Row(overall=3.0, count(1)=16357),
 Row(overall=4.0, count(1)=37445),
 Row(overall=5.0, count(1)=102790)]

In [36]:
reviews_df.count() - sum(output.collect()[i][1] for i in range(5))

0

In [37]:
type(reviews_df)

pyspark.sql.dataframe.DataFrame

Convert to RDD!

In [38]:
reviews_df.rdd

MapPartitionsRDD[120] at javaToPython at NativeMethodAccessorImpl.java:0

In [39]:
type(reviews_df.rdd)

pyspark.rdd.RDD

### Count the words in the first row

In [40]:
row_one = df.first()

In [41]:
row_one

Row(asin='0439893577', helpful=[0, 0], overall=5.0, reviewText='I like the item pricing. My granddaughter wanted to mark on it but I wanted it just for the letters.', reviewTime='01 29, 2014', reviewerID='A1VXOAVRGKGEAK', reviewerName='Angie', summary='Magnetic board', unixReviewTime=1390953600)

In [42]:
def word_count(text):
    return len(text.split())

In [43]:
word_count(row_one['reviewText'])

20

In [44]:
from pyspark.sql.types import IntegerType

#'udf' is for User Defined Function!

word_count_udf = F.udf(word_count, returnType=IntegerType())

In [45]:
review_text_col = df['reviewText']

In [46]:
counts_df = df.withColumn('wordCount', word_count_udf(review_text_col))

In [47]:
# Remember that we set the default number of lines to show at 5.

show(counts_df).T

,0,1,2,3,4
asin,0439893577,0439893577,0439893577,0439893577,0439893577
helpful,"[0, 0]","[1, 1]","[1, 1]","[0, 0]","[1, 1]"
overall,5,4,5,5,4
reviewText,I like the item pricing. My granddaughter want...,Love the magnet easel... great for moving to d...,Both sides are magnetic. A real plus when you...,Bought one a few years ago for my daughter and...,I have a stainless steel refrigerator therefor...
reviewTime,"01 29, 2014","03 28, 2014","01 28, 2013","02 8, 2014","05 5, 2014"
reviewerID,A1VXOAVRGKGEAK,A8R62G708TSCM,A21KH420DK0ICA,AR29QK6HPFYZ4,ACCH8EOML6FN5
reviewerName,Angie,Candace,capemaychristy,dcrm,DoyZ
summary,Magnetic board,it works pretty good for moving to different a...,love this!,Daughters love it,Great to have so he can play with his alphabet...
unixReviewTime,1390953600,1395964800,1359331200,1391817600,1399248000
wordCount,20,22,76,31,47


In [48]:
from pyspark.sql.types import IntegerType
word_count_udf = F.udf(word_count, IntegerType())

# Registering our word_count() function so that we
# can use it with SQL! See documentation here:
# https://jaceklaskowski.gitbooks.io/mastering-spark-sql/spark-sql-UDFRegistration.html

df.createOrReplaceTempView('reviews')
spark.udf.register('word_count', word_count_udf)

<function __main__.word_count(text)>

In [49]:
query = """
SELECT asin, overall, reviewText, word_count(reviewText) AS wordCount
FROM reviews
"""

In [50]:
counts_df = spark.sql(query)

In [51]:
show(counts_df)

,asin,overall,reviewText,wordCount
0,0439893577,5.0,I like the item pricing. My granddaughter want...,20
1,0439893577,4.0,Love the magnet easel... great for moving to d...,22
2,0439893577,5.0,Both sides are magnetic. A real plus when you...,76
3,0439893577,5.0,Bought one a few years ago for my daughter and...,31
4,0439893577,4.0,I have a stainless steel refrigerator therefor...,47


In [52]:
def count_all_the_things(text):
    return [len(text), len(text.split())]

In [53]:
from pyspark.sql.types import ArrayType, IntegerType
count_udf = F.udf(count_all_the_things, returnType=ArrayType(IntegerType()))

In [54]:
counts_df = df.withColumn('counts', count_udf(df['reviewText']))

In [55]:
show(counts_df, 1)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,counts
0,0439893577,"[0, 0]",5.0,I like the item pricing. My granddaughter want...,"01 29, 2014",A1VXOAVRGKGEAK,Angie,Magnetic board,1390953600,"[100, 20]"


In [56]:
slim_counts_df = (
    df.drop('reviewTime', 'helpful')
#       .drop('helpful')
      .withColumn('counts', count_udf(df['reviewText']))
      .drop('reviewText')
)

In [57]:
show(slim_counts_df, n=1)

,asin,overall,reviewerID,reviewerName,summary,unixReviewTime,counts
0,0439893577,5.0,A1VXOAVRGKGEAK,Angie,Magnetic board,1390953600,"[100, 20]"


In [58]:
aggs = counts_df.groupBy('reviewerID').agg({'overall': 'mean'})
aggs.collect()

[Row(reviewerID='A3R9X003XW0LNR', avg(overall)=4.36),
 Row(reviewerID='A2TZT5AMK1XQTC', avg(overall)=4.2),
 Row(reviewerID='A2UGBQSNS1I37M', avg(overall)=4.857142857142857),
 Row(reviewerID='AUAX1QWUCYKSX', avg(overall)=4.090909090909091),
 Row(reviewerID='A1XU4ZI0NUZ68M', avg(overall)=3.8333333333333335),
 Row(reviewerID='A2OIGPPBTR65MR', avg(overall)=4.421052631578948),
 Row(reviewerID='A2B2OT465SEI15', avg(overall)=4.833333333333333),
 Row(reviewerID='A3RSDKOJ28VU6B', avg(overall)=5.0),
 Row(reviewerID='A8SHD09Z1RI3', avg(overall)=4.0),
 Row(reviewerID='A1919BYRFSG18U', avg(overall)=4.666666666666667),
 Row(reviewerID='A10AKE9TAADHVV', avg(overall)=4.142857142857143),
 Row(reviewerID='A239YR4IQ0QAFY', avg(overall)=5.0),
 Row(reviewerID='A1VLE2SH9J8WYS', avg(overall)=4.785714285714286),
 Row(reviewerID='A24WSD48Z3WUZS', avg(overall)=4.5),
 Row(reviewerID='A17A1KTVI3DG6U', avg(overall)=3.5714285714285716),
 Row(reviewerID='A2GF3I3279PEXO', avg(overall)=4.714285714285714),
 Row(reviewe

### A few more basic commands

Please refer also to the [official programming guide](http://spark.apache.org/docs/latest/rdd-programming-guide.html).

In [59]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)

In [60]:
distData.

SyntaxError: invalid syntax (<ipython-input-60-8d5f5b508eb1>, line 1)

In [63]:
distData

ParallelCollectionRDD[166] at parallelize at PythonRDD.scala:195

In [61]:
def multiply(a, b):
    return a * b

In [62]:
distData.reduce(multiply)

120

In [64]:
def subtract1(a, b):
    return a - b

In [65]:
distData.reduce(subtract1)

-13

In [66]:
def subtract2(a, b):
    return b - a

In [67]:
distData.reduce(subtract2)

3

Can you explain these "subtraction" results?

In [68]:
distData.filter(lambda x: x < 4).collect()

[1, 2, 3]

### Reading files

```sc.textFile()``` for .txt files

`.toJSON()` for .json files

In [69]:
dfjson = counts_df.toJSON()

In [70]:
df2 = spark.read.json(dfjson)

In [71]:
df2.printSchema()

root
 |-- asin: string (nullable = true)
 |-- counts: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [72]:
counts_df

DataFrame[asin: string, helpful: array<bigint>, overall: double, reviewText: string, reviewTime: string, reviewerID: string, reviewerName: string, summary: string, unixReviewTime: bigint, counts: array<int>]

In [73]:
type(df.toPandas())

pandas.core.frame.DataFrame